In [15]:
!pip install pydantic

In [16]:
!pip install langchain-google-genai

In [13]:
!playwright install


In [26]:
!pip install -qU "langchain[google-genai]"

^C


In [1]:
import os
from typing import Final
from telegram import Update
from telegram.ext import ContextTypes
from dotenv import load_dotenv
import nest_asyncio
nest_asyncio.apply()

In [2]:
load_dotenv()
TELEGRAM_BOT_TOKEN: Final = os.getenv("TOKEN")
TELEGRAM_BOT_USERNAME: Final = os.getenv("BOT_USERNAME")
API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from CommunicationAgent import Communication

# Instantiate Gemini model with config
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # latest identifier
    google_api_key= API_KEY,
    temperature=0.7,
    max_tokens=1024,
)
#create prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a friendly, knowledgeable ecommerce assistant named Sahayak. 
            Your role is to help users discover, compare, and understand products available online, especially budget-friendly 
            options in India. You speak clearly and concisely, using everyday language. Provide helpful suggestions and answer 
            questions related to categories like fashion, electronics, home goods, and deals. Always guide users politely, and 
            if you don't know something or lack real-time access, recommend searching online. Never give financial or medical advice. 
            Use emojis sparingly to make responses more engaging. Stay factual and avoid personal opinions.
     """),
    ("human", "{input}")
])
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=300,  # Customize based on your model's context window
    memory_key="chat_history",
    input_key="input",
    summary_prompt="summarize the response["text"]he user and his/her preferences from past messages"
)
chain = LLMChain(llm=llm, prompt=prompt, memory=memory)
# Async function to interact with Gemini using LangChain
def askAi(content: str) -> dict:
    response = chain.invoke(input=content)
    print(memory.chat_memory.messages)
    answer = Communication(response['text'])
    return 

SyntaxError: invalid syntax. Perhaps you forgot a comma? (965466707.py, line 30)

##Commands

In [4]:
async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """
    Sends a greeting message when the /start command is issued.

    Args:
        update (Update): Incoming update from Telegram.
        context (ContextTypes.DEFAULT_TYPE): Context object with metadata and bot data.
    """
    await update.message.reply_text(
        "Hello, give me a description of the product you want to search and let's get going!!!!"
    )

In [5]:
async def help_command (update: Update, context: ContextTypes.DEFAULT_TYPE):
    """sends a message to user when confused

    Args:
        update (Update): _description_
        context (ContextTypes.DEFAULT_TYPE): _description_
    """
    await update.message.reply_text("Give me a description of the product you want to buy to get started.")

##Responses

In [10]:
from CommunicationAgent import Communication
import asyncio
def handle_response (text: str) -> str:
    """handle responses

    Args:
        text (str): _description_

    Returns:
        str: _description_
    """
    # response = askAi(text)
    response = asyncio.run(Communication(text))
    # print('Our Response: ', response)
    return response

In [ ]:
# conversation_history = []

In [7]:
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """handling messages

    Args:
        update (Update): _description_
        context (ContextTypes.DEFAULT_TYPE): _description_
    """
    message_type: str = update.message.chat.type
    text: str = update.message.text
    
    print(f'user ({update.message.chat.id}) in {message_type}: "{text}"')
    
    if message_type == 'group':
        if TELEGRAM_BOT_USERNAME in text:
            new_text: str = text.replace(TELEGRAM_BOT_USERNAME, '').strip()
            # conversation_history.append(new_text)
            response: str = handle_response(new_text)
            # conversation_history.append(response)
        else:
            return
    else:
        response: str = handle_response(text)
    
    #print('Bot: ', response)
    
    await update.message.reply_text(response)

In [8]:
async def error(update: Update, context: ContextTypes.DEFAULT_TYPE):
    print(f'Update {update} caused error: {context.error}')
    

In [11]:
from telegram.ext import Application, CommandHandler, MessageHandler, filters
print('starting bot....')
app = Application.builder().token(TELEGRAM_BOT_TOKEN).build()

#commands
app.add_handler(CommandHandler('start', start_command))
app.add_handler(CommandHandler('help', help_command))

#messsages
app.add_handler(MessageHandler(filters.TEXT, handle_message))

#errors
app.add_error_handler(error)

#polls the bot
print("polling...")
app.run_polling(poll_interval=3)

starting bot....
polling...
INFO     [telegram.ext.Application] Application started
user (6111544138) in private: "hello myself is ayush, 
i want to see the Motorola phones under 15k  from flipkart which are available on zip/pin code = 303108"
INFO     [browser_use.agent.service] 💾 File system path: C:\Users\BIT\AppData\Local\Temp\browser_use_agent_06873587-2c9b-748f-8000-ccec5b381f6c
INFO     [browser_use.Agent🅰 1f6c on 🆂 1f6c 🅟 96] 🧠 Starting a browser-use agent 0.5.4 with base_model=gemini-2.5-flash +vision extraction_model=gemini-2.5-flash +file_system
INFO     [browser_use.Agent🅰 1f6c on 🆂 1f6c 🅟 96] 🚀 Starting task: hello myself is ayush, 
i want to see the Motorola phones under 15k  from flipkart which are available on zip/pin code = 303108


Task exception was never retrieved
future: <Task finished name='Task-54' coro=<Connection.run() done, defined at c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_connection.py:303> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\BIT\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_connection.py", line 310, in run
    await self._transport.connect()
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

WARNING  [bubus.helpers] setup_playwright failed (attempt 3/4): NotImplementedError: . Waiting 1.0s before retry...


Task exception was never retrieved
future: <Task finished name='Task-63' coro=<Connection.run() done, defined at c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_connection.py:303> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\BIT\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_connection.py", line 310, in run
    await self._transport.connect()
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

ERROR    [bubus.helpers] setup_playwright failed after 4 attempts over 3.6s. Semaphore wait: 0.1s. Final error: NotImplementedError: 
WARNING  [browser_use.Agent🅰 1f6c on 🆂 1f6c 🅟 96] Full state retrieval failed: NotImplementedError: 
WARNING  [browser_use.Agent🅰 1f6c on 🆂 1f6c 🅟 96] 🔄 Falling back to minimal state summary


...

Task exception was never retrieved
future: <Task finished name='Task-165' coro=<Connection.run() done, defined at c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_connection.py:303> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\BIT\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_connection.py", line 310, in run
    await self._transport.connect()
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

WARNING  [bubus.helpers] setup_playwright failed (attempt 3/4): NotImplementedError: . Waiting 1.0s before retry...


Task exception was never retrieved
future: <Task finished name='Task-195' coro=<Connection.run() done, defined at c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_connection.py:303> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\BIT\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_connection.py", line 310, in run
    await self._transport.connect()
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 133, in connect
    raise exc
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\playwright\_impl\_transport.py", line 120, in connect
    self._proc = await asyncio.create_subprocess_exec(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

ERROR    [bubus.helpers] setup_playwright failed after 4 attempts over 3.0s. Semaphore wait: 0.0s. Final error: NotImplementedError: 
ERROR    [browser_use.Agent🅰 1f6c on 🆂 1f6c 🅟 96] ❌ Result failed 3/3 times:
 
INFO     [browser_use.Agent🅰 1f6c on 🆂 1f6c 🅟 96] 📍 Step 1: Ran 1 actions in 6.27s: ✅ 1
ERROR    [browser_use.Agent🅰 1f6c on 🆂 1f6c 🅟 96] ❌ Stopping due to 3 consecutive failures
None
INFO     [telegram.ext.Application] Application is stopping. This might take a moment.
CRITICAL [telegram.ext.Application] Fetching updates was aborted due to KeyboardInterrupt(). Suppressing exception to ensure graceful shutdown.
Traceback (most recent call last):
  File "c:\Users\BIT\OneDrive\Desktop\Walmart\.venv\Lib\site-packages\telegram\ext\_application.py", line 671, in stop
    self.__update_fetcher_task.result()
  File "C:\Users\BIT\AppData\Local\Programs\Python\Python312\Lib\asyncio\futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "C:\

RuntimeError: Cannot close a running event loop